In [12]:
import requests # for making standard html requests
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup # magical tool for parsing html data
import json # for parsing data
from pandas import DataFrame as df # premier library for data organization
import csv
import random
import time

In [13]:
# Build list of user agents to avoid being blocked.
user_agents = ["Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0",
               "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
               "Mozilla/5.0"]
containers = []

# Pull 10 pages of listings for list of suburbs in Cork city area.
for location in ['douglas','ballincollig','carrigaline','bishopstown','glasheen','blackrock',
                'wilton','model-farm-road']:
    for offset in range(0,60,20): # 20 listings per page
        user_agent = random.sample(user_agents,1)
        url = "http://www.daft.ie/cork-city/houses-for-sale/" + location + "/?offset=" + str(offset)
        req = Request(url , headers={'User-Agent': str(user_agent)})
        webpage = urlopen(req).read()
        page_soup = soup(webpage, "html.parser")
        containers += page_soup.findAll("div","AdCard__adCardContainer AdCard__adCardContainer--premium")
        containers += page_soup.findAll("div","AdCard__adCardContainer AdCard__adCardContainer--standard")
        time.sleep(5) # Wait 5 seconds between hitting site
        print(location,offset, len(containers))

douglas 0 18
douglas 20 38
douglas 40 58
ballincollig 0 75
ballincollig 20 93
ballincollig 40 93
carrigaline 0 110
carrigaline 20 130
carrigaline 40 142
bishopstown 0 160
bishopstown 20 165
bishopstown 40 165
glasheen 0 180
glasheen 20 180
glasheen 40 180
blackrock 0 198
blackrock 20 202
blackrock 40 202
wilton 0 205
wilton 20 205
wilton 40 205
model-farm-road 0 217
model-farm-road 20 217
model-farm-road 40 217


In [14]:
# For each kind of listing, create a list of containers.
# containers = page_soup.findAll("div","AdCard__adCardContainer AdCard__adCardContainer--premium")
containers += page_soup.findAll("div","AdCard__adCardContainer AdCard__adCardContainer--standard")

# Write data to an array for output to csv later
rows = []

for idx, container in enumerate(containers):
    listing = {}
    # Check if the listing is "valid", ads will not have a price section.
    if container.find('strong', "PropertyInformationCommonStyles__costAmountCopy") is None:
        print("Ad Detected!")
    else: 
        listing['listing_id'] = container.find('a',"PropertyInformationCommonStyles__addressCopy--link")["href"].split("-")[-1][:-1]
        listing['location'] = container.find('a',"PropertyInformationCommonStyles__propertyPrice--link")["href"].split("/")[3]
        listing['address'] = container.find('a',"PropertyInformationCommonStyles__addressCopy--link").getText().replace(",","|")
        listing['beds'] = container.find('div', "QuickPropertyDetails__iconCopy").getText()
        listing['bathrooms'] = container.find('div', "QuickPropertyDetails__iconCopy--WithBorder").getText()
        listing['cost'] = container.find('strong', "PropertyInformationCommonStyles__costAmountCopy").getText().replace("€","")
        listing['num_pics'] = container.find('span', "PropertyImage__picturesAmountCopy").getText()
        listing['property_type'] = container.find('div', "QuickPropertyDetails__propertyType").getText().replace("\n","").strip()
        try:
            listing['ber'] = container.find("img",{"class":"PropertyImage__berImage"}).get_attribute_list("src")[0][-6:-4]
        except:
            listing['ber'] = 'NA'
        rows.append(list(listing.values())) # Add listing to rows list for output.

Ad Detected!
Ad Detected!
Ad Detected!
Ad Detected!
Ad Detected!


In [15]:
len(rows)

212

In [16]:
# Output to flat file for modeling 
#filename = "cork_property_prices.csv"
filename = "cork_property_prices_" + str(round(time.time())) + ".csv"

# field names  
fields = ['listing_id', 'location', 'address', 'beds', 'bathrooms', 'cost', 'num_pics', 'property_type', 'ber'] 

# writing to csv file  
with open(filename, 'w') as csvfile:  
    # creating a csv writer object  
    csvwriter = csv.writer(csvfile)  
        
    # writing the fields  
    csvwriter.writerow(fields)  
        
    # writing the data rows  
    csvwriter.writerows(rows) 